In [1]:
%pip install torchvision
%pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall
import matplotlib.pyplot as plt
from torchvision import datasets, models
import torchvision.transforms as transforms
from collections import Counter
from PIL import Image

In [21]:
torch.manual_seed(0)
print(torch.cuda.is_available())


True


In [4]:
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(
    'archive/seg_train/seg_train', transform= transform)
dataloader_train = DataLoader(train_dataset, shuffle=True,batch_size= 32, num_workers=4)
print(train_dataset.classes)
print(train_dataset.class_to_idx)

test_dataset = datasets.ImageFolder(
    'archive/seg_test/seg_test', transform= transform
)
dataloader_test = DataLoader(test_dataset, shuffle=True, batch_size= 32, num_workers=4)
print(test_dataset.classes)
print(test_dataset.class_to_idx)

print(dict(Counter(train_dataset.targets)))


['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
{'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}
['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
{'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}
{0: 2191, 1: 2271, 2: 2404, 3: 2512, 4: 2274, 5: 2382}


In [5]:
#Look at the data
image, label = next(iter(dataloader_train))
print(image.shape)
image = image.squeeze(0)
print("Shape after squeezing channel dimension:", image.shape)

image = image.permute(1, 2, 0)
plt.imshow(image)
plt.show()



torch.Size([32, 3, 150, 150])
Shape after squeezing channel dimension: torch.Size([32, 3, 150, 150])


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 4 is not equal to len(dims) = 3

In [23]:
#Building the Neural Network
class Intel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.mod = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size= 3, padding= 1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size= 2),
            nn.Conv2d(32, 64, kernel_size= 3, padding= 1),
            nn.ReLU(),
            #nn.Dropout(0.2),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size= 2),
            nn.Conv2d(64, 128, kernel_size= 3, padding= 1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size= 2),
            nn.Flatten(),
            #nn.Dropout(0.3),
        )
        self.fc1 = nn.Linear(128 * 18 * 18, 128)
        self.fc2 = nn.Linear(128, num_classes)
        
    
    def forward(self, x):
        x = self.mod(x)
        x = self.fc1(x) 
        return self.fc2(x)

inte = Intel(num_classes= 6)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(inte.parameters(), lr=0.002)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
inte.to(device)

for epoch in range(5):
        for images , labels in dataloader_train:
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = inte(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            

            

In [24]:
met_accuracy = Accuracy(task='multiclass', num_classes= 6).to(device)
metric_precision = Precision( task="multiclass", num_classes=6, average="macro").to(device)
metric_recall = Recall( task="multiclass", num_classes=6, average="macro").to(device)
inte.eval()

with torch.no_grad():
    for images, labels in dataloader_test:
        images = images.to(device)
        labels = labels.to(device)
        outputs = inte(images)
        _, preds = torch.max(outputs, dim=1)
        
        met_accuracy.update(preds, labels)
        metric_precision.update(preds, labels)
        metric_recall.update(preds, labels)
    accuracy = met_accuracy.compute()
    precision = metric_precision.compute()
    recall = metric_recall.compute()
# test commit
print(accuracy)
print(precision)
print(recall)

tensor(0.5700, device='cuda:0')
tensor(0.6229, device='cuda:0')
tensor(0.5728, device='cuda:0')


# Now we test the accuracy of pre-trained models
# Resnet 50

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for i in [0.003]: #0.003
    #initalize model and freeze layers while adding our own
    res = models.resnet50(weights= 'IMAGENET1K_V1')
    for param in res.parameters():
        param.requires_grad = False

    num_ftrs = res.fc.in_features

    res.fc = nn.Sequential (nn.Linear(num_ftrs, 128),
                            nn.Linear(128, 64),
                            nn.Linear(64, 6),
                            )

    res = res.to(device)



    #initalize optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(res.fc.parameters(), lr= i)
    
    #train in epochs
    for epoch in range(10):
        print('EPOCH {}:'.format(epoch + 1))
        for images , labels in dataloader_train:
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = res(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
                
    #setup metrics
    met_accuracy = Accuracy(task='multiclass', num_classes= 6).to(device)
    metric_precision = Precision( task="multiclass", num_classes=6, average="macro").to(device)
    metric_recall = Recall( task="multiclass", num_classes=6, average="macro").to(device)
    
    #evaluate our model on test
    res.eval()

    with torch.no_grad():
        for images, labels in dataloader_test:
            images = images.to(device)
            labels = labels.to(device)
            outputs = res(images)
            _, preds = torch.max(outputs, dim=1)
            
            met_accuracy.update(preds, labels)
            metric_precision.update(preds, labels)
            metric_recall.update(preds, labels)
        accuracy = met_accuracy.compute()
        precision = metric_precision.compute()
        recall = metric_recall.compute()
        
    # test commit
    print('for ', i, ' : ' ,accuracy)
    print(precision)
    print(recall)

EPOCH 1:
EPOCH 2:
EPOCH 3:
EPOCH 4:
EPOCH 5:
EPOCH 6:
EPOCH 7:
EPOCH 8:
EPOCH 9:
EPOCH 10:
for  0.003  :  tensor(0.8870, device='cuda:0')
tensor(0.8950, device='cuda:0')
tensor(0.8907, device='cuda:0')


In [19]:
ttt = Image.open('./gla.jpg')
ttt = transform(ttt)
ttt = ttt.unsqueeze(0)
ttt = ttt.to(device)

with torch.no_grad():  # Disable gradient computation
    output = res(ttt)

# Get the predicted class
_, predicted = torch.max(output, 1)
print(f'Predicted class: {predicted.item()}')

Predicted class: 2


In [6]:
import pickle

model_pkl_file = "intel_img_resnet50.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(res, file)
    
 

In [7]:
   
loaded_res = pickle.load(open("intel_img_resnet50.pkl"  , 'rb'))    

met_accuracy = Accuracy(task='multiclass', num_classes= 6).to(device)
metric_precision = Precision( task="multiclass", num_classes=6, average="macro").to(device)
metric_recall = Recall( task="multiclass", num_classes=6, average="macro").to(device)
loaded_res.eval()

with torch.no_grad():
    for images, labels in dataloader_test:
        images = images.to(device)
        labels = labels.to(device)
        outputs = loaded_res(images)
        _, preds = torch.max(outputs, dim=1)
        
        met_accuracy.update(preds, labels)
        metric_precision.update(preds, labels)
        metric_recall.update(preds, labels)
    accuracy = met_accuracy.compute()
    precision = metric_precision.compute()
    recall = metric_recall.compute()
# test commit
print(accuracy)
print(precision)
print(recall)

tensor(0.8700, device='cuda:0')
tensor(0.8761, device='cuda:0')
tensor(0.8730, device='cuda:0')


# VGG19

In [13]:

vgg = models.vgg19(weights='IMAGENET1K_V1')
for param in vgg.parameters():
    param.requires_grad = False

#num_ftrs = vgg.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
vgg.classifier[6] = nn.Linear(vgg.classifier[6].in_features, 6)

vgg = vgg.to(device)



for i in [0.003]: #0.003
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(vgg.classifier[6].parameters(), lr= i)
    for epoch in range(5):
        print('EPOCH {}:'.format(epoch + 1))
        for images , labels in dataloader_train:
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = vgg(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
                
                
    met_accuracy = Accuracy(task='multiclass', num_classes= 6).to(device)
    metric_precision = Precision( task="multiclass", num_classes=6, average="macro").to(device)
    metric_recall = Recall( task="multiclass", num_classes=6, average="macro").to(device)
    vgg.eval()

    with torch.no_grad():
        for images, labels in dataloader_test:
            images = images.to(device)
            labels = labels.to(device)
            outputs = vgg(images)
            _, preds = torch.max(outputs, dim=1)
            
            met_accuracy.update(preds, labels)
            metric_precision.update(preds, labels)
            metric_recall.update(preds, labels)
        accuracy = met_accuracy.compute()
        precision = metric_precision.compute()
        recall = metric_recall.compute()
        
    # test commit
    print('for ', i, ' : ' ,accuracy)
    print(precision)
    print(recall)

EPOCH 1:
EPOCH 2:
EPOCH 3:
EPOCH 4:
EPOCH 5:
for  0.003  :  tensor(0.8710, device='cuda:0')
tensor(0.8754, device='cuda:0')
tensor(0.8739, device='cuda:0')


In [14]:
model_pkl_file_2 = "intel_img_vgg.pkl"  

with open(model_pkl_file_2, 'wb') as file:  
    pickle.dump(vgg, file)
    
 